In [6]:
import os
import deepspeed
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch


!pip install torch==1.11.0 --extra-index-url https://download.pytorch.org/whl/cu113
!pip install transformers==4.20.1
!pip install deepspeed==0.6.5


In [8]:
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [2]:
local_rank = int(os.getenv('LOCAL_RANK', '0'))
world_size = int(os.getenv('WORLD_SIZE', '1'))


In [9]:
model = AutoModelForCausalLM.from_pretrained(
    "EleutherAI/gpt-j-6B", revision="float16", torch_dtype=torch.float16, low_cpu_mem_usage=True
)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")


Downloading:   0%|          | 0.00/836 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/11.3G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.94k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/357 [00:00<?, ?B/s]

In [10]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)


In [11]:
op = pipe(
     """[Text]: Helena Smith founded Core.ai 2 years ago. She is now the CEO and CTO of the company and is building a team of highly skilled developers in machine learning and natural language processing.
        [Position]: CEO and CTO
        ###
        [Text]: Tech Robotics is a robot automation company specialized in AI driven robotization. Its Chief Technology Officer, Max Smith, says a new wave of improvements should be expected for next year.
        [Position]: Chief Technology Officer
        ###
        [Text]: François is a Go developer. He mostly works as a freelancer but is open to any kind of job offering!
        [Position]: Go developer
        ###
        [Text]: Chitti designs dresses, she is a fashion designer and a great singer
        [Position]: fashion designer
        ###
        [Text]: Maxime is a bio-scientist at Auto Dataset, and he's been working there for 1 year.
        [Position]: bio-scientist
        ### 
        Text]: Jo is a Java Developer. He mostly works as a freelancer but is open to any kind of job offering!
        [Position]:""",
    do_sample=False,
    max_sample =500,
    max_length=350
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [12]:
print(op[0]['generated_text'])

[Text]: Helena Smith founded Core.ai 2 years ago. She is now the CEO and CTO of the company and is building a team of highly skilled developers in machine learning and natural language processing.
        [Position]: CEO and CTO
        ###
        [Text]: Tech Robotics is a robot automation company specialized in AI driven robotization. Its Chief Technology Officer, Max Smith, says a new wave of improvements should be expected for next year.
        [Position]: Chief Technology Officer
        ###
        [Text]: François is a Go developer. He mostly works as a freelancer but is open to any kind of job offering!
        [Position]: Go developer
        ###
        [Text]: Chitti designs dresses, she is a fashion designer and a great singer
        [Position]: fashion designer
        ###
        [Text]: Maxime is a bio-scientist at Auto Dataset, and he's been working there for 1 year.
        [Position]: bio-scientist
        ### 
        Text]: Jo is a Java Developer. He mostly works

In [82]:
op = pipe("""
        [Text] : A 63-year-old woman with no known cardiac history presented with a sudden onset of dyspnea requiring intubation and ventilatory support out of hospital. 
        [Symptom] : woman
        ###
        [Text] : A 28-year-old previously healthy man presented with a 6-week history of palpitations. The symptoms occurred during rest, 2–3 times per week, lasted up to 30 minutes at a time and were associated with dyspnea.
        [Symptom] : man
        ###
        [Text] : 23 year old young man with a history of smoking was admitted with chest pain
        [Symptom] : man
        ###
        [Text] : 37 yo F with history of depression and suicidal attempt in the past presented with obtundation
        [Symptom] : female
        ###
        [Text] : 72 year old female with ESLD [**12-27**] HCV cirrhosis admitted to MICU with AMS and sepsis
        [Symptom] :  female
        ###
        [Text] :69-year-old patient with a history of T1, N0, squamous cell carcinoma of the left true vocal cord,
        who presented with respiratory distress
        [Symptom] : None 
        ###
        [Text] : 72 year old man with cirrhosis admitted with fever
        [Symptom] : """,
    
    do_sample=False,
    max_sample =700,
    max_length=450
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [83]:
print(op[0]['generated_text'])


        [Text] : A 63-year-old woman with no known cardiac history presented with a sudden onset of dyspnea requiring intubation and ventilatory support out of hospital. 
        [Symptom] : woman
        ###
        [Text] : A 28-year-old previously healthy man presented with a 6-week history of palpitations. The symptoms occurred during rest, 2–3 times per week, lasted up to 30 minutes at a time and were associated with dyspnea.
        [Symptom] : man
        ###
        [Text] : 23 year old young man with a history of smoking was admitted with chest pain
        [Symptom] : man
        ###
        [Text] : 37 yo F with history of depression and suicidal attempt in the past presented with obtundation
        [Symptom] : female
        ###
        [Text] : 72 year old female with ESLD [**12-27**] HCV cirrhosis admitted to MICU with AMS and sepsis
        [Symptom] :  female
        ###
        [Text] :69-year-old patient with a history of T1, N0, squamous cell carcinoma of the left 

In [84]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Sat Mar 11 18:57:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/

In [108]:
op = pipe("""
        [Text] : A 63-year-old woman with no known cardiac history presented with a sudden onset of dyspnea requiring intubation and ventilatory support out of hospital. 
        [Age] : 63
        [History] : no known cardiac history
        ###
        [Text] : A 28-year-old previously healthy man presented with a 6-week history of palpitations. The symptoms occurred during rest, 2–3 times per week, lasted up to 30 minutes at a time and were associated with dyspnea.
        [Age] : 28
        [History] : previously healthy 
        ###
        [Text] : 23 year old young man with a history of smoking was admitted with chest pain.
        [Age] : 23
        [History] : history of smoking
        ###
        [Text] : 37 yo F with history of depression and suicidal attempt in the past presented with obtundation
        [Age] : 37
        [History] : history of depression and suicidal attempt
        ###
        [Text] : 72 year old female with ESLD [**12-27**] HCV cirrhosis admitted to MICU with AMS and sepsis
        [Age] : 72
        [History] :  HCV cirrhosis
        ###
        [Text] :69-year-old patient with a history of T1, N0, squamous cell carcinoma of the left true vocal cord,
        who presented with respiratory distress
        [Age] : 69
        [History] : history of T1, N0, squamous cell carcinoma 
        ###
        [Text] : 62 yo F with h/o, HTN, hypertensive heart disease, who presents with two days of fever, nausea,
        vomiting and mental status changes.""",
    
    do_sample=False,
    max_sample =700,
    max_length= 600
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [109]:
print(op[0]['generated_text'])


        [Text] : A 63-year-old woman with no known cardiac history presented with a sudden onset of dyspnea requiring intubation and ventilatory support out of hospital. 
        [Age] : 63
        [History] : no known cardiac history
        ###
        [Text] : A 28-year-old previously healthy man presented with a 6-week history of palpitations. The symptoms occurred during rest, 2–3 times per week, lasted up to 30 minutes at a time and were associated with dyspnea.
        [Age] : 28
        [History] : previously healthy 
        ###
        [Text] : 23 year old young man with a history of smoking was admitted with chest pain.
        [Age] : 23
        [History] : history of smoking
        ###
        [Text] : 37 yo F with history of depression and suicidal attempt in the past presented with obtundation
        [Age] : 37
        [History] : history of depression and suicidal attempt
        ###
        [Text] : 72 year old female with ESLD [**12-27**] HCV cirrhosis admitted to 